Data for the fingerprints depending on latitud and longitud

In [2]:
function get_fingerprints()

    fp_dir = joinpath(@__DIR__, "..", "data")
    isdir(fp_dir) || mkpath(fp_dir)
    fp_file = joinpath(fp_dir, "FINGERPRINTS_SLANGEN_Bakker.nc")
    if !isfile(fp_file)
        url = "https://github.com/scrim-network/BRICK/raw/master/fingerprints/FINGERPRINTS_SLANGEN_Bakker.nc"
        download(url, fp_file)
    end

    fplat = ncread(fp_file,"lat")
    fplon = ncread(fp_file,"lon")
    fpAIS = ncread(fp_file,"AIS")
    fpGSIC = ncread(fp_file,"GLAC")
    fpGIS = ncread(fp_file,"GIS")
    ncclose()

    return fplat,fplon,fpAIS,fpGSIC,fpGIS
end

get_fingerprints (generic function with 1 method)

In [123]:
using NetCDF
(fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()


([-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5, -80.5  …  80.5, 81.5, 82.5, 83.5, 84.5, 85.5, 86.5, 87.5, 88.5, 89.5], [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5  …  350.5, 351.5, 352.5, 353.5, 354.5, 355.5, 356.5, 357.5, 358.5, 359.5], Float32[NaN NaN … 1.1086327 1.1114359; NaN NaN … 1.108638 1.1114365; … ; NaN NaN … 1.1086223 1.1114353; NaN NaN … 1.1086272 1.1114354], Float32[NaN NaN … 0.021808445 0.09062562; NaN NaN … 0.022879047 0.091268495; … ; NaN NaN … 0.019840943 0.08925989; NaN NaN … 0.020792248 0.08995592], Float32[NaN NaN … -1.5367132 -1.3215039; NaN NaN … -1.5308045 -1.3199075; … ; NaN NaN … -1.5482318 -1.3246047; NaN NaN … -1.5425231 -1.3230699])

The matrices of fpAIS, fpGSIC, and fpGIS, are constructed in the form of [Lon,Lat], therefore we want to nearest to or segment. The functions minimumlat(x) and minimumlon(x) help us to find this values.

In [49]:
function minimumlat(x)
    Distance=zeros(1,180);
    for i in 1:180
        distance1=(x-fplat[i])^2;
        Distance[i]=distance1;
    end
    m=minimum(Distance)
    RowNumber=findall(r->r==m,Distance);
    return(RowNumber)
end

minimumlat (generic function with 1 method)

In [54]:
function minimumlon(x)
    Distance=zeros(1,360);
    for i in 1:360
        distance1=(x-fplon[i])^2;
        Distance[i]=distance1;
    end
    m=minimum(Distance)
    RowNumber=findall(r->r==m,Distance);
    return(RowNumber)
end

minimumlon (generic function with 1 method)

Segments selected Manzanillo,Colima : lat 19.1138 lon -104.3385
                  New Orlands: lat 29.9511 lon -90.0715

In [52]:
#Latitude selected for the segment
minimumlat(19.1138)


1-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 110)

In [56]:
#Longitud selected for the sectment
lon1=mod(-104.3385,360)
minimumlon(lon1)

1-element Vector{CartesianIndex{2}}:
 CartesianIndex(1, 256)

Fingerprint value

In [105]:
isnan(fpGSIC[256,110])


true

In [62]:
fpAIS[256,110]*AIS+fpGIS[256,110]*GIS+fpGSIC*GLAC+TE+LWS

UndefVarError: UndefVarError: AIS_RCP not defined

Code to obtain the components of global sea level rise

In [110]:
import Pkg 

In [114]:
Pkg.add("CSV")

In [125]:
using DataFrames
using CSV
Brickcomps = CSV.File("/Users/ce3304/Downloads/projections_MAP_RCP85_sneasybrick.csv") |> DataFrame

Row,YEAR,GMSL,LWS,GLAC,GIS,AIS,TE,TEMP,OCHEAT,CO2,OCEANCO2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?
1,1850.0,-0.102153,0.000325643,-0.0307438,0.00305963,-0.0489542,0.00727428,-0.181525,-18.8794,279.192,0.740886
2,1851.0,-0.0684015,0.000326355,-0.0272493,0.00290103,-0.042466,0.00766773,-0.0688389,-5.6435,278.942,-2.30122
3,1852.0,-0.0955456,0.000264489,-0.030078,0.00243201,-0.0412545,0.00804689,-0.156925,-34.7849,279.874,-1.57303
4,1853.0,-0.0599957,-1.31477e-5,-0.0251425,0.00275303,-0.0407208,0.00840254,-0.114708,-30.7725,282.065,-1.06167
5,1854.0,-0.0836821,0.000269126,-0.0264085,0.00268931,-0.0407323,0.00872585,0.227197,-21.4257,282.073,-1.00631
6,1855.0,-0.0115779,0.000469725,-0.0253438,0.0025343,-0.0409695,0.00890898,-0.0987867,-29.2714,282.778,0.836196
7,1856.0,-0.0800312,0.000438501,-0.0273048,0.00248161,-0.039903,0.00859525,-0.0233152,-23.576,282.104,-0.84116
8,1857.0,-0.0835417,0.000261133,-0.0264662,0.00237941,-0.0406448,0.00773374,-0.337109,-42.0525,282.789,0.454979
9,1858.0,-0.0649237,0.000713304,-0.0258677,0.0028196,-0.0400596,0.0070105,-0.217046,-47.2591,282.447,0.641629


Vectors of all values for (btime,AIS,GSIC,GIS,TE,LWS,GMSL)

In [150]:
btime=Brickcomps[:,1];
GMSL=Brickcomps[:,2];
LWS=Brickcomps[:,3];
GIS= Brickcomps[:,5];
GSIC=Brickcomps[:,4];
TE=Brickcomps[:,7];
AIS=Brickcomps[:,6];


Tuple for brickcomps

In [148]:
brickcomps=Tuple([btime,AIS,GSIC,GIS,TE,LWS,GMSL]);

Downscale function

In [146]:
using NetCDF
function downscale_brick(brickcomps, lonlat, ensInds, ystart, yend, tstep)
    # To do - check with vectors of lat, lon
    (fplat,fplon,fpAIS,fpGSIC,fpGIS) = get_fingerprints()
    (btime,AIS,GSIC,GIS,TE,LWS,GMSL) = brickcomps

    # Select indices of time of interest, with respect to timestep
    tinds = findall(x -> x .>= ystart && x .<=yend, btime)
    years = collect(ystart:yend)
    yinds = findall(x -> x % tstep==0, years)
    # Need to normalize LSL relative to 2000
    inorm = findall(x -> x==2000, btime)

    tdim=length(btime)

    if length(years)==length(tinds)
        tinds = tinds[yinds]
    else
        println("Error: years outside of bounds")
        return nothing
    end

    num_ens = length(ensInds)

    # Output matrix: ens x time x segment
    lsl_out = zeros(num_ens, length(tinds), length(lonlat))

    # Trim component vectors to timesteps and ensembles. Assume interval is 1 year
    if tdim==size(AIS)[1] # check that time dimension is 1
        # for normalizing
        AIS_norm = AIS[inorm,ensInds]
        GSIC_norm = GSIC[inorm,ensInds]
        GIS_norm = GIS[inorm,ensInds]
        TE_norm = TE[inorm,ensInds]
        LWS_norm = LWS[inorm,ensInds]
        GMSL_norm = GMSL[inorm,ensInds]
        # actual projections
        AIS = AIS[tinds,ensInds]
        GSIC = GSIC[tinds,ensInds]
        GIS = GIS[tinds,ensInds]
        TE = TE[tinds,ensInds]
        LWS = LWS[tinds,ensInds]
        GMSL = GMSL[tinds,ensInds]
    else
        println("Error: time dimension is not 1 for brick components")
        return nothing
    end

    for f in 1:length(lonlat) # Loop through lonlat tuples

        lon = lonlat[f][1]
        lat = lonlat[f][2]
        # Convert Longitude to degrees East
        # CIAM Lat is already in (-90,90) by default
        if lon <0
            lon = lon + 360
        end

        # Find fingerprint degrees nearest to lat,lon
        ilat = findall(isequal(minimum(abs.(fplat.-lat))),abs.(fplat.-lat))
        ilon = findall(isequal(minimum(abs.(fplon.-lon))),abs.(fplon.-lon))


        # Take average of closest lat/lon values
        fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[ilon,ilat])))
        fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[ilon,ilat])))
        fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[ilon,ilat])))

        fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
        fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
        fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]
        fpTE_loc = 1.0
        fpLWS_loc=1.0

        # Keep searching nearby lat/lon values if fingerprint value is NaN unless limit is hit
        inc =1

        while isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc) && inc<5

            newlonStart = lon_subtractor.(fplon[ilon],inc)[1]
            newlatStart = lat_subtractor.(fplat[ilat],inc)[1]
            newlonEnd = lon_adder.(fplon[ilon],inc)[1]
            newlatEnd = lat_adder.(fplat[ilat],inc)[1]

            latInd1 = minimum(findall(isequal(minimum(abs.(fplat.-newlatStart))),abs.(fplat.-newlatStart)))
            #minimum(findall(x-> x in newlatStart,fplat))
            latInd2 = maximum(findall(isequal(minimum(abs.(fplat.-newlatEnd))),abs.(fplat.-newlatEnd)))
            #maximum(findall(x -> x in newlatEnd,fplat))

            lonInd1 = minimum(findall(isequal(minimum(abs.(fplon.-newlonStart))),abs.(fplon.-newlonStart)))
            #minimum(findall(x-> x in newlonStart,fplon))
            lonInd2 = maximum(findall(isequal(minimum(abs.(fplon.-newlonEnd))),abs.(fplon.-newlonEnd)))
            #maximum(findall(x -> x in newlonEnd,fplon))

            if latInd2 < latInd1
                latInds=[latInd1; 1:latInd2]
            else
                latInds=latInd1:latInd2
            end

            if lonInd2 < lonInd1
                lonInds=[lonInd1; 1:lonInd2]
            else
                lonInds = lonInd1:lonInd2
            end

            fpAIS_flat = collect(skipmissing(Iterators.flatten(fpAIS[lonInds,latInds])))
            fpGSIC_flat = collect(skipmissing(Iterators.flatten(fpGSIC[lonInds,latInds])))
            fpGIS_flat = collect(skipmissing(Iterators.flatten(fpGIS[lonInds,latInds])))

            fpAIS_loc = mean(fpAIS_flat[isnan.(fpAIS_flat).==false],dims=1)[1]
            fpGSIC_loc = mean(fpGSIC_flat[isnan.(fpGSIC_flat).==false],dims=1)[1]
            fpGIS_loc = mean(fpGIS_flat[isnan.(fpGIS_flat).==false],dims=1)[1]

            inc = inc + 1

        end

        # If still NaN, throw an error
        if isnan(fpAIS_loc) || isnan(fpGIS_loc) || isnan(fpGSIC_loc)
            println("Error: no fingerprints found for ($(lon),$(lat))")
            return nothing
        end

       # Multiply fingerprints by BRICK ensemble members
       if ndims(AIS) > 1
            for n in 1:size(AIS)[2] # loop through ensemble members
                lsl_out[n, :, f] = fpGIS_loc * GIS[:,n] + fpAIS_loc * AIS[:,n] + fpGSIC_loc * GSIC[:,n] +
                                   fpTE_loc * TE[:,n] + fpLWS_loc * LWS[:,n]
                # CIAM - LSL should be sea-level change relative to year 2000
                lsl_norm = fpGIS_loc * GIS_norm[n] + fpAIS_loc * AIS_norm[n] + fpGSIC_loc * GSIC_norm[n] +
                           fpTE_loc * TE_norm[n] + fpLWS_loc * LWS_norm[n]
                lsl_out[n, :, f] = lsl_out[n, :, f] .- lsl_norm
            end
        else
            lsl_out[1, :, f] = fpGIS_loc * GIS[:] + fpAIS_loc * AIS[:] + fpGSIC_loc * GSIC[:] +
                fpTE_loc * TE[:] + fpLWS_loc * LWS[:]
            # CIAM - LSL should be sea-level change relative to year 2000
            lsl_norm = fpGIS_loc * GIS_norm + fpAIS_loc * AIS_norm + fpGSIC_loc * GSIC_norm +
                       fpTE_loc * TE_norm + fpLWS_loc * LWS_norm
            lsl_out[1, :, f] = lsl_out[1, :, f] .- lsl_norm
        end

    end # End lonlat tuple

    return lsl_out,GMSL
end

downscale_brick (generic function with 4 methods)

In [149]:
ensInds=[1];
lonlat= (-104.3385,19.1138);
ystart=2010;
yend=2200;
tstep=10;
downscale_brick(brickcomps, lonlat, ensInds, ystart, yend, tstep)

BoundsError: BoundsError